In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import mdshare
import pyemma

In [ ]:
pdb = mdshare.fetch('pentapeptide-impl-solv.pdb', working_directory='data')
files = mdshare.fetch('pentapeptide-*-500ns-impl-solv.dcd', working_directory='data')

In [ ]:
torsions_feat = pyemma.coordinates.featurizer(pdb)
torsions_feat.add_backbone_torsions(cossin=True)
torsions_data = pyemma.coordinates.load(files, features=torsions_feat)
labels = ['backbone\ntorsions']

positions_feat = pyemma.coordinates.featurizer(pdb)
positions_feat.add_selection(positions_feat.select_Backbone())
positions_data = pyemma.coordinates.load(files, features=positions_feat)
labels += ['backbone atom\npositions']

distances_feat = pyemma.coordinates.featurizer(pdb)
distances_feat.add_distances(
    distances_feat.pairs(distances_feat.select_Backbone(), excluded_neighbors=2))
distances_data = pyemma.coordinates.load(files, features=distances_feat)
labels += ['backbone atom\ndistances']

In [ ]:
dim = 10

fig, axes = plt.subplots(1, 3, figsize=(12, 3), sharey=True)
for ax, lag in zip(axes.flat, [5, 10, 20]):
    torsions_vamp = pyemma.coordinates.vamp(torsions_data, lag=lag, dim=dim)
    scores = [torsions_vamp.score()]
    positions_vamp = pyemma.coordinates.vamp(positions_data, lag=lag, dim=dim)
    scores += [positions_vamp.score()]
    distances_vamp = pyemma.coordinates.vamp(distances_data, lag=lag, dim=dim)
    scores += [distances_vamp.score()]
    ax.bar(labels, scores)
    ax.set_title(r'lag time $\tau$={:.1f}ns'.format(lag * 0.1))
    if lag == 5:
        vamp_bars_plot = dict(labels=labels, scores=scores, dim=dim, lag=lag) # save for later
axes[0].set_ylabel('VAMP2 score')
fig.tight_layout()

In [ ]:
lags = [1, 2, 5, 10, 20, 50]
dims = [i + 1 for i in range(10)]
vamp_conv_plot = dict(scores=[], lags=lags, dims=dims) # save for later

fig, ax = plt.subplots()
for lag in lags:
    scores = [pyemma.coordinates.vamp(torsions_data, lag=lag, dim=dim).score()
              for dim in dims]
    ax.plot(dims, scores, label='lag={:.1f}ns'.format(lag * 0.1))
    vamp_conv_plot['scores'].append(scores)
ax.legend()
ax.set_xlabel('number of dimensions')
ax.set_ylabel('VAMP2 score')
fig.tight_layout()

#TODO: Explain why we chose lag 5

We perform TICA at the optimal using the lag time obtained from the VAMP-2 score.

In [ ]:
tica = pyemma.coordinates.tica(torsions_data, lag=5)
tica_output = tica.get_output()
tica_all = np.concatenate(tica_output)

We visualize the marginal and joint distributions of our TICA components by simple histograming. 

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
pyemma.plots.plot_feature_histograms(tica_all, ax=axes[0])
pyemma.plots.plot_density(*tica_all.T[:2], ax=axes[1], logscale=True)
fig.tight_layout()

Let’s have a look how one of the trajectories looks like in the space of the first $4$ TICA components. We can see that the TICA components nicely resolve the slow transitions as discrete jumps.

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(12, 5), sharex=True)
x = 0.1 * np.arange(tica_output[0].shape[0])
for i, (ax, tic) in enumerate(zip(axes.flat, tica_output[0].T)):
    ax.plot(x, tic)
    ax.set_ylabel('IC {}'.format(i + 1))
axes[-1].set_xlabel('time / ns')
fig.tight_layout()

The TICA coordinates are now clustered into a number of discrete states using the $k$-means algorithm. The $k$-means algorithm requires as input the number of clusters. The trajectories are automatically assigned to the cluster centers by calling `cluster.dtrajs`. 

In [ ]:
cluster = pyemma.coordinates.cluster_kmeans(tica_output, k=200, max_iter=50, stride=10)
dtrajs_all = np.concatenate(cluster.dtrajs)

We can check the location of our discrete states by plotting them onto the density of our data in the first two TICA dimensions:

In [ ]:
fig, ax = plt.subplots()
pyemma.plots.plot_density(*tica_all.T[:2], ax=ax, cbar=False, alpha=0.3)
ax.scatter(*cluster.clustercenters.T[:2], s=5, c='C1')
ax.set_xlabel('IC 1')
ax.set_ylabel('IC 2')
fig.tight_layout()

The states are well distributed in this space.


Now, we calculate the implied timescales with `pyemma.msm.its()` up to a lagtime of $50$ steps (as defined by `lags=50`). Instead of a single number, an array can be passed to compute the ITS at defined lagtimes. We directly estimate the error bars with Bayesian MSMs. To speed up the computation, this can be disabled.

In [ ]:
its = pyemma.msm.its(cluster.dtrajs, lags=50, nits=10, errors='bayes')
pyemma.plots.plot_implied_timescales(its, units='ns', dt=0.1);

The implied timescales converge quickly; above $0.5$ ns the slowest ones are constant within the error. We thus select a lag time of $5$ steps ($0.5$ ns) to build a Markov model. The lagtime to estimate the Markov model is specified as `lag` here. As a quick check we print the fraction of states and counts that are in the active set.

In [ ]:
msm = pyemma.msm.bayesian_markov_model(cluster.dtrajs, lag=5, dt_traj='0.1 ns')
print('fraction of states used = {:f}'.format(msm.active_state_fraction))
print('fraction of counts used = {:f}'.format(msm.active_count_fraction))

The model is validated with a Chapman-Kolmogorov test. The number of metastable sets to test for is estimated from the implied timescales plot. We find four  processes that can be resolved up to $2.5$ ns. To capture those, we need to define $5$ metastable states. This assumption is validated with the Chapman-Kolmogorov test.

In [ ]:
nstates = 5
cktest = msm.cktest(nstates)
pyemma.plots.plot_cktest(cktest);

The assumption of $5$ metastable states seems to hold.

From the MSM which is now stored in the object we called `msm`, various properties can be obtained. We start by analyzing the stationary distribution and the free energy computed over the first two TICA coordinates. The stationary distribution $\pi$ is stored in `msm.pi` or `msm.stationary_distribution`. We further compute the free energy landscape by re-weighting the trajectory frames with stationary probabilities from the MSM.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharex=True, sharey=True)
pyemma.plots.plot_contour(
    *tica_all.T[:2],
    msm.pi[dtrajs_all],
    ax=axes[0],
    mask=True,
    cbar_label='stationary distribution')
pyemma.plots.plot_free_energy(
    *tica_all.T[:2],
    weights=np.concatenate(msm.trajectory_weights()),
    ax=axes[1],
    legacy=False)
for ax in axes.flat:
    ax.set_xlabel('IC 1')
axes[0].set_ylabel('IC 2')
fig.tight_layout()

Now we analyze the slowest processes by looking at the distribution of states along the first 3 eigenvectors.

In [ ]:
eigvec = msm.eigenvectors_right()
print('first eigenvector is one: {} (min={}, max={})'.format(
    np.allclose(eigvec[:, 0], 1, atol=1e-15), eigvec[:, 0].min(), eigvec[:, 0].max()))

fig, axes = plt.subplots(1, 3, figsize=(12, 3))
for i, ax in enumerate(axes.flat):
    pyemma.plots.plot_contour(
        *tica_all.T[:2],
        eigvec[dtrajs_all, i + 1],
        ax=ax,
        cmap='PiYG',
        cbar_label='{}. right eigenvector'.format(i + 2),
        mask=True)
    ax.set_xlabel('IC 1')
axes[0].set_ylabel('IC 2')
fig.tight_layout()

From the sign change of the eigenvectors we extract between which states the corresponding process happens. Since the eigenvectors were internally sorted according to their eigenvalue, they correspond to the slowest processes of the implied timescale plot.

Next, we do a coarse graining into a user defined number of macrostates. As already discussed, $4$ is a good choice for this example.

In [ ]:
msm.pcca(nstates);

We have now determined the probability for each microstate to belong to a given macrostate. These probabilities are called memberships to a given macrostate.

In [ ]:
fig, axes = plt.subplots(1, msm.n_metastable, figsize=(15, 3), sharex=True, sharey=True)
for i, ax in enumerate(axes.flat):
    pyemma.plots.plot_contour(
        *tica_all.T[:2],
        msm.metastable_distributions[i][dtrajs_all],
        ax=ax,
        cmap='afmhot_r', 
        mask=True,
        method='nearest',
        cbar_label='metastable distribution {}'.format(i))
    ax.set_xlabel('IC 1')
axes[0].set_ylabel('IC 2')
fig.tight_layout()

Let's see how PCCA has coarse grained our state space in the first two TICA projections.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

metastable_traj = msm.metastable_assignments[dtrajs_all]

pyemma.plots.plot_state_map(*tica_all.T[:2], metastable_traj, ax=ax, zorder=-1)

ax.set_xlabel('IC 1')
ax.set_ylabel('IC 2')
ax.set_xlim(-2, 8)
ax.set_ylim(-2, 8)
fig.tight_layout()

For each macrostate we can generate a number of representative sample structures and store them into a trajectory file.

In [ ]:
pcca_samples = msm.sample_by_distributions(msm.metastable_distributions, 10)
torsions_source = pyemma.coordinates.source(files, features=torsions_feat)
pyemma.coordinates.save_trajs(
    torsions_source,
    pcca_samples,  
    outfiles=['./data/pcca{}_10samples.xtc'.format(n) for n in range(msm.n_metastable)])

In [ ]:
#TODO: visualize metastable structures with nglview

Since we now have a coarse representation of our data, let's extract some information from it. We start with the stationary distribution which encodes the relative free energy of the states. It can be computed by 
$$
\Delta G = - k_b T \ln(pi)
$$

In [ ]:
print('state\tπ\t\tΔG/kT')
for i, s in enumerate(msm.metastable_sets):
    p = msm.pi[s].sum()
    print('{}\t{:f}\t{:f}'.format(i, p, -np.log(p)))

Knowing PCCA metastable states, we can also extract mean first passage times (MFPTs) and transition rates between them. They can be simply printed...

In [ ]:
mfpt = np.zeros((nstates, nstates))
for i in range(nstates):
    for j in range(nstates):
        mfpt[i, j] = msm.mfpt(
            msm.metastable_sets[i],
            msm.metastable_sets[j])
        
rate = np.zeros_like(mfpt)
nz = mfpt.nonzero()
rate[nz] = 1.0 / mfpt[nz]

print('MFPT / ns')
print('from/to\t|' + '_______'.join([str(n) for n in range(msm.n_metastable)]))
print('\n'.join([str(n) + '\t|'+'\t'.join(np.round(m, 1).astype(str)) for n, m in enumerate(mfpt)]))

... or displayed in a network plot. Here, we scale the arrows according to the transition rates and annotate them with MFPTs.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
pyemma.plots.plot_network(
   rate,
   pos=np.asarray([[np.cos(x + 0.5), np.sin(x + 0.5)] for x in np.linspace(0, 2 * np.pi, msm.n_metastable, endpoint=False)]),
   figpadding=0.2,
   state_scale=0.3,
   arrow_label_format='%.1f ns',
   arrow_labels=mfpt,
   arrow_scale=0.5,
   size=4,
   ax=ax);

Further, the flux between metastable states can be computed and coarse grained as follows. We chose metastable states $4$ and $3$ as an example between which the flux is computed.

In [ ]:
A = msm.metastable_sets[4]
B = msm.metastable_sets[3]
flux = pyemma.msm.tpt(msm, A, B)

cg, cgflux = flux.coarse_grain(msm.metastable_sets)

The committor as projected in the first two TICA can be displayed with a filled countour plot.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

pyemma.plots.plot_contour(
    *tica_all.T[:2], flux.committor[np.concatenate(cluster.dtrajs)], cmap='brg', ax=ax,
    mask=True, method='nearest', cbar_label='committor 0 -> 3', alpha=0.8, zorder=-1);

ax.set_xlim(-2, 8)
ax.set_ylim(-2, 8)
fig.tight_layout()

We find that the committor is constant within the metastable sets defined above. Transition regions can be identified by committor values $\tilde{} 0.5$.

In [ ]:
#hmm = pyemma.msm.bayesian_hidden_markov_model(cluster.dtrajs, 4)

In [ ]:
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import LogLocator
from matplotlib.cm import get_cmap

#mpl.rc('font', **{'family':'serif', 'serif':['Computer Modern Roman']})
mpl.rcParams['axes.titlesize'] = 8
mpl.rcParams['axes.labelsize'] = 8
mpl.rcParams['font.size'] = 8
mpl.rcParams['legend.fontsize'] = 5
mpl.rcParams['xtick.labelsize'] = 5
mpl.rcParams['ytick.labelsize'] = 5
mpl.rcParams['text.usetex'] = False
mpl.rcParams['xtick.minor.pad'] = 3
mpl.rcParams['xtick.major.pad'] = 3
mpl.rcParams['ytick.minor.pad'] = 3
mpl.rcParams['ytick.major.pad'] = 3
mpl.rcParams['axes.labelpad'] = 1
mpl.rcParams['lines.markersize'] = 4

In [ ]:
fig = plt.figure(figsize=(3.47, 4))
gw = int(np.floor(0.5 + 1000 * fig.get_figwidth()))
gh = int(np.floor(0.5 + 1000 * fig.get_figheight()))
gs = plt.GridSpec(gh, gw)
gs.update(hspace=0.0, wspace=0.0, left=0.0, right=1.0, bottom=0.0, top=1.0)

ax_box = fig.add_subplot(gs[:, :])
ax_box.set_axis_off()
ax_box.text(0.00, 0.97, '(a)', size=10)
ax_box.text(0.00, 0.55, '(b)', size=10)
ax_box.text(0.55, 0.97, '(c)', size=10)
ax_box.text(0.00, 0.30, '(d)', size=10)

ax_feat = fig.add_subplot(gs[200:1300, 400:1800])
ax_feat.bar(vamp_bars_plot['labels'], vamp_bars_plot['scores'], 0.5)
ax_feat.tick_params(axis='x', labelrotation=35)
ax_feat.set_ylabel('VAMP2 score')
ax_feat.set_title(r'lag time $\tau$={:.1f}ns'.format(vamp_bars_plot['lag'] * 0.1))

ax_density = fig.add_subplot(gs[400:1550, 2200:3350])
_, _, misc = pyemma.plots.plot_density(
    *tica_all.T[:2],
    ax=ax_density,
    cax=fig.add_subplot(gs[300:350, 2200:3350]),
    cbar_orientation='horizontal',
    logscale=True)
misc['cbar'].set_ticks(LogLocator(base=10.0))
misc['cbar'].ax.xaxis.set_ticks_position('top')
misc['cbar'].ax.xaxis.set_label_position('top')
ax_density.scatter(*cluster.clustercenters.T[:2], s=0.5, c='C1', marker='p')
ax_density.set_xlabel('IC 1')
ax_density.set_ylabel('IC 2')

x = 0.1 * np.arange(tica_output[0].shape[0])
ax_tic1 = fig.add_subplot(gs[1850:2200, 400:3350])
ax_tic2 = fig.add_subplot(gs[2200:2550, 400:3350])

ax_tic1.plot(x, tica_output[0][:, 0])
ax_tic2.plot(x, tica_output[0][:, 1])
ax_tic1.set_ylabel('IC 1')
ax_tic2.set_ylabel('IC 2')
ax_tic2.set_xlabel('time / ns')

ax_its = fig.add_subplot(gs[2870:3670, 400:3350])
pyemma.plots.plot_implied_timescales(its, units='ns', dt=0.1, ax=ax_its, nits=4, ylog=False)
ax_its.set_ylim(1, 15)
ax_its.set_xlabel(r'lag time $\tau$ / ns')

fig.savefig('figure_1.pdf', dpi=300)

In [ ]:
fig, axes = pyemma.plots.plot_cktest(cktest, figsize=[3.47, 3.47], dt=0.1, units='ns')
for ax in axes[-1, :]:
    ax.set_xlabel('$\tau$ / ns')
for ax in axes[:, 0]:
    ax.set_ylabel('prob.')
fig.savefig('figure_2.pdf', dpi=300)

In [ ]:
fig = plt.figure(figsize=(3.47, 4))
gw = int(np.floor(0.5 + 1000 * fig.get_figwidth()))
gh = int(np.floor(0.5 + 1000 * fig.get_figheight()))
gs = plt.GridSpec(gh, gw)
gs.update(hspace=0.0, wspace=0.0, left=0.0, right=1.0, bottom=0.0, top=1.0)

ax_box = fig.add_subplot(gs[:, :])
ax_box.set_axis_off()
ax_box.text(0.00, 0.97, '(a)', size=10)
ax_box.text(0.52, 0.97, '(b)', size=10)
ax_box.text(0.00, 0.50, '(c)', size=10)
ax_box.text(0.52, 0.50, '(d)', size=10)

ax_fe = fig.add_subplot(gs[400:1750, 400:1750])
_, _, misc = pyemma.plots.plot_free_energy(
    *tica_all.T[:2],
    weights=np.concatenate(msm.trajectory_weights()),
    ax=ax_fe,
    cax=fig.add_subplot(gs[300:350, 400:1750]),
    cbar_orientation='horizontal',
    legacy=False)
misc['cbar'].set_ticks(np.linspace(0, 8, 5))
misc['cbar'].ax.xaxis.set_ticks_position('top')
misc['cbar'].ax.xaxis.set_label_position('top')
#ax_fe.set_xlabel('IC 1')
ax_fe.set_ylabel('IC 2')

ax_state = fig.add_subplot(gs[400:1750, 2000:3350])
_, _, misc = pyemma.plots.plot_state_map(
    *tica_all.T[:2],
    metastable_traj,
    ax=ax_state,
    cax=fig.add_subplot(gs[300:350, 2000:3350]),
    cbar_label='metastable state',
    cbar_orientation='horizontal')
misc['cbar'].ax.xaxis.set_ticks_position('top')
misc['cbar'].ax.xaxis.set_label_position('top')
ax_state.set_xticklabels([])
ax_state.set_yticklabels([])

#ax_mfpt = fig.add_subplot(gs[-1900:, :1900])
#pyemma.plots.plot_network(
#    rate,
#    pos=np.asarray([[np.cos(x + 0.5), np.sin(x + 0.5)] for x in np.linspace(0, 2 * np.pi, 5, endpoint=False)]),
#    figpadding=0.2,
#    state_scale=0.3,
#    state_colors=get_cmap(cmap, nstates).colors,
#    arrow_label_format='%.1f ns',
#    arrow_labels=mfpt,
#    arrow_scale=0.5,
#    size=4,
#    ax=ax_mfpt)


evec_idx = 1
ax_eig = fig.add_subplot(gs[2300:3650, 400:1750], zorder=1)
_, _, misc = pyemma.plots.plot_contour(
    *tica_all.T[:2],
    eigvec[dtrajs_all, evec_idx],
    cmap='PiYG',
    ax=ax_eig,
    mask=True,
    cax=fig.add_subplot(gs[2200:2250, 400:1750]),
    cbar_label='{}. right eigenvector'.format(evec_idx + 1),
    cbar_orientation='horizontal')
misc['cbar'].set_ticks(np.linspace(*misc['cbar'].get_clim(), 3))
misc['cbar'].ax.xaxis.set_ticks_position('top')
misc['cbar'].ax.xaxis.set_label_position('top')
ax_eig.set_xlabel('IC 1')
ax_eig.set_ylabel('IC 2')

ax_flux = fig.add_subplot(gs[2300:3650, 2000:3350], zorder=1)
_, _, misc = pyemma.plots.plot_contour(
    *tica_all.T[:2],
    flux.committor[np.concatenate(cluster.dtrajs)],
    cmap='brg',
    ax=ax_flux,
    mask=True,
    cax=fig.add_subplot(gs[2200:2250, 2000:3350]),
    cbar_label=r'committor 4 $\to$ 3',
    cbar_orientation='horizontal')
misc['cbar'].set_ticks(np.linspace(0, 1, 3))
misc['cbar'].ax.xaxis.set_ticks_position('top')
misc['cbar'].ax.xaxis.set_label_position('top')
ax_flux.set_xlabel('IC 1')
#ax_flux.set_ylabel('IC 2')

fig.savefig('figure_3.pdf', dpi=300)